<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/selectivity_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
new_mnist_trainset =  [ [[],[]] for i in range(10)]
new_mnist_testset  =  [ [[],[]] for i in range(10)]

for i in range(60000):
    for j in range(10):
        if mnist_trainset[i][1] == j:
            # image 
            new_mnist_trainset[j][0].append(mnist_trainset[i][0])  
            # label
            new_mnist_trainset[j][1].append(mnist_trainset[i][1])

for i in range(10000):
    for j in range(10):
        if mnist_testset[i][1] == j:
            # image 
            new_mnist_testset[j][0].append(mnist_testset[i][0])  
            # label
            new_mnist_testset[j][1].append(mnist_testset[i][1])

image_trainset = list()
label_trainset = list()

image_testset = list()
label_testset = list()

for i in range(10):
    image_trainset.append(new_mnist_trainset[i][0])
    label_trainset.append(new_mnist_trainset[i][1])

for i in range(10):
    image_testset.append(new_mnist_testset[i][0])
    label_testset.append(new_mnist_testset[i][1])

flattened_image_train = list()
flattened_label_train = list()

flattened_image_test = list()
flattened_label_test = list()

# flattening image 
for sublist in image_trainset:
    for val in sublist:
        flattened_image_train.append(val)

# flattening label
for sublist in label_trainset:
    for val in sublist:
        flattened_label_train.append(val)

# flattening image 
for sublist in image_testset:
    for val in sublist:
        flattened_image_test.append(val)

# flattening label
for sublist in label_testset:
    for val in sublist:
        flattened_label_test.append(val)

flattened_image_train = torch.stack(flattened_image_train)
flattened_label_train = torch.Tensor(flattened_label_train)
flattened_label_train = flattened_label_train.type(torch.LongTensor)

flattened_image_test = torch.stack(flattened_image_test)
flattened_label_test = torch.Tensor(flattened_label_test)
flattened_label_test = flattened_label_test.type(torch.LongTensor)

train_dataset = TensorDataset(flattened_image_train, flattened_label_train)
train_dataloader = DataLoader(train_dataset, batch_size=50)

test_dataset = TensorDataset(flattened_image_test, flattened_label_test)
test_dataloader = DataLoader(test_dataset, batch_size=50)

In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 100
def selectivity_trainer(optimizer, model):

    hidden_layer_each_neuron = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
    hidden_layer_each_neuron = np.array(hidden_layer_each_neuron)

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

            # Print/Append activation of the hidden layer 
            # print(model.layer_activations.shape)
            # model.layer_activations

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # find selectivity at the final epoch 
            if epoch == no_epochs - 1: # last epoch 
                for activation, label in zip(model.layer_activations, labels):
                    # shape of activation and label: 256 and 1 
                    
                    # get the actual value of item. This is because label is now Tensor 
                    label = label.item()

                    # this is not part of gradient calculcation 
                    with torch.no_grad():
                        activation = activation.numpy()

                    # for each image/label, append activation value of neuron 
                    for i in range(256):    # number of neurons in hidden layer 
                        hidden_layer_each_neuron[i][label].append(activation[i])


        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
        #     print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
        #     torch.save(model.state_dict(), "model.dth")

    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # selectivity_list contains all of the selectivity of each neuron 
    selectivity_list = list()

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity_list
        selectivity_list.append(selectivity)

    return test_acc, selectivity_list

# AdaDelta

In [ ]:
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, Adadelta_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("sorted_selectivity_Adadelta.txt", "a")
f.write(str(0)+'\n'+str(Adadelta_test_acc)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')
f.close()

!cp sorted_selectivity_Adadelta.txt /content/drive/MyDrive

# AdaGrad

In [19]:
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, Adagrad_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("sorted_selectivity_Adagrad.txt", "w")
f.write(str(0)+'\n'+str(Adagrad_test_acc)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')
f.close()

!cp sorted_selectivity_Adagrad.txt /content/drive/MyDrive

model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.13168899, Test Loss: 7.70306222, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.09843616, Test Loss: 5.65562728, Test Accuracy: 0.10090000

Epoch: 3/100, Train Loss: 0.06130542, Test Loss: 4.59363047, Test Accuracy: 0.11980000

Epoch: 4/100, Train Loss: 0.04320877, Test Loss: 4.18983584, Test Accuracy: 0.16600000

Epoch: 5/100, Train Loss: 0.03890744, Test Loss: 3.68634688, Test Accuracy: 0.23830000

Epoch: 6/100, Train Loss: 0.03450411, Test Loss: 3.44733186, Test Accuracy: 0.28380000

Epoch: 7/100, Train Loss: 0.03204197, Test Loss: 3.35837846, Test Accuracy: 0.31280000

Epoch: 8/100, Train Loss: 0.02985216, Test Loss: 3.19376002, Test Accuracy: 0.32760000

Epoch: 9/100, Train Loss: 0.02999220, Test Loss: 3.15467740, Test Accuracy: 0.33590000

Epoch: 10/100, Train Loss: 

# SGD


In [20]:
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, SGD_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("sorted_selectivity_SGD.txt", "w")
f.write(str(0)+'\n'+str(SGD_test_acc)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')
f.close()

!cp sorted_selectivity_SGD.txt /content/drive/MyDrive

model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.07133562, Test Loss: 8.03251186, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.09449442, Test Loss: 7.71593376, Test Accuracy: 0.10090000

Epoch: 3/100, Train Loss: 0.10823328, Test Loss: 7.35257972, Test Accuracy: 0.10090000

Epoch: 4/100, Train Loss: 0.10545864, Test Loss: 6.66556555, Test Accuracy: 0.10090000

Epoch: 5/100, Train Loss: 0.08940878, Test Loss: 5.83873217, Test Accuracy: 0.10090000

Epoch: 6/100, Train Loss: 0.07399541, Test Loss: 5.22182636, Test Accuracy: 0.10490000

Epoch: 7/100, Train Loss: 0.06473352, Test Loss: 4.81070747, Test Accuracy: 0.12790000

Epoch: 8/100, Train Loss: 0.05982259, Test Loss: 4.51134584, Test Accuracy: 0.15470000

Epoch: 9/100, Train Loss: 0.05700355, Test Loss: 4.27659487, Test Accuracy: 0.17710000

Epoch: 10/100, Train Loss: 0.05

# Adam

In [21]:
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("sorted_selectivity_Adam.txt", "w")
f.write(str(0)+'\n'+str(Adam_test_acc)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')
f.close()

!cp sorted_selectivity_Adam.txt /content/drive/MyDrive

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 1.07672777, Test Loss: 4.39992207, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.54172696, Test Loss: 4.00939839, Test Accuracy: 0.12280000

Epoch: 3/100, Train Loss: 0.21198839, Test Loss: 3.79308346, Test Accuracy: 0.23870000

Epoch: 4/100, Train Loss: 0.14705217, Test Loss: 3.92706918, Test Accuracy: 0.28540000

Epoch: 5/100, Train Loss: 0.11832885, Test Loss: 4.23987269, Test Accuracy: 0.28970000

Epoch: 6/100, Train Loss: 0.10747726, Test Loss: 4.83581270, Test Accuracy: 0.28510000

Epoch: 7/100, Train Loss: 0.09820665, Test Loss: 4.42070674, Test Accuracy: 0.34850000

Epoch: 8/100, Train Loss: 0.08706567, Test Loss: 4.50853499, Test Accuracy: 0.32840000

Epoch: 9/100, Train Loss: 0.07909970, Test Loss: 4.78619497, Test Accuracy: 0.33760000

Epoch: 10/100, Train Loss: 0.0